# Topic Modelling

Creating a system which, given turns of conversation that has happened and a new turn, can return the most important topic(s) the conversation is revolving around

Note: This will be fed into the knowledge retreival phases to provide the core context in addition to the latest turn of conversation

### Imports